<a href="https://colab.research.google.com/github/mvdheram/Stereotypical-Social-bias-detection-/blob/Pre-trained-LM-selection-and-training/Huggingface_XLNET_GPT_2_Multi_label.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Mon Aug 23 23:33:53 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install transformers==4.5.1 --quiet

     |████████████████████████████████| 2.1 MB 7.8 MB/s 
     |████████████████████████████████| 895 kB 64.8 MB/s 
     |████████████████████████████████| 3.3 MB 52.2 MB/s 


In [3]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import (AutoTokenizer, AutoModelForSequenceClassification, 
                          PreTrainedModel,
                          TrainingArguments, Trainer)
from transformers import XLNetTokenizer, XLNetForSequenceClassification
from transformers import GPT2Tokenizer, GPT2Model, GPT2ForSequenceClassification
from torch.utils.data import Dataset , DataLoader
from transformers.modeling_outputs import SequenceClassifierOutput
from transformers import AdamW
from tqdm import trange
import pandas as pd
import numpy as np
from torch.nn import BCEWithLogitsLoss
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report
import logging
import argparse
from tqdm import tqdm
from torch import nn

# Loading 

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df_ohe = pd.read_csv('/content/drive/MyDrive/Trained_models/mult_label_dataset/Unrelated_samples_adjusted/ohe_multilabel.csv', index_col = 0)

In [6]:
y = df_ohe.iloc[:,:-1].values
X = df_ohe.iloc[:,-1].values

In [7]:
MAX_LEN = 50
RANDOM_SEED = 47

In [8]:
from sklearn.model_selection import train_test_split

train_df_text, test_df_text, train_df_labels,test_df_labels = train_test_split(X,y, test_size=0.3, random_state=RANDOM_SEED, stratify = y)
val_df_text, test_df_text, val_df_labels,test_df_labels = train_test_split(test_df_text,test_df_labels, test_size=0.5, random_state=RANDOM_SEED,stratify = test_df_labels)

In [9]:
LABEL_COLUMNS = ['Ethnicity',	'gender'	,'profession'	,'religion',	'Anti-stereotype',	'stereotype',	'unrelated']

In [10]:
train_df_labels = pd.DataFrame(train_df_labels, columns= LABEL_COLUMNS)
val_df_labels = pd.DataFrame(val_df_labels, columns= LABEL_COLUMNS)
test_df_labels = pd.DataFrame(test_df_labels, columns= LABEL_COLUMNS)
train_df_text = pd.DataFrame(train_df_text, columns = ['sentence'])
val_df_text = pd.DataFrame(val_df_text, columns = ['sentence'])
test_df_text = pd.DataFrame(test_df_text, columns = ['sentence'])

In [11]:
train_df = pd.concat([train_df_text,train_df_labels],axis=1)
val_df = pd.concat([val_df_text,val_df_labels], axis = 1)
test_df = pd.concat([test_df_text,test_df_labels], axis = 1)

In [12]:
train_df.shape, val_df.shape, test_df.shape

((11592, 8), (2484, 8), (2484, 8))

In [13]:
train_df

,sentence,Ethnicity,gender,profession,religion,Anti-stereotype,stereotype,unrelated
0,although to now there is no evidence of an ord...,0.0,0.0,0.0,1.0,0.0,1.0,0.0
1,The movie I also saw last night in had a plot ...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,The mathematician works at a university. He is...,0.0,1.0,0.0,0.0,0.0,1.0,0.0
3,why does germany never have fun and always wor...,0.0,0.0,0.0,1.0,0.0,1.0,0.0
4,they say strength in numbers tell that to mill...,0.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...
11587,It and is brighter at night falls when today t...,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11588,I smell flowers outside.,0.0,0.0,0.0,0.0,0.0,0.0,1.0
11589,The immigrant from yemen thought he was too li...,1.0,0.0,0.0,0.0,1.0,0.0,0.0
11590,Dinosaurs aren't real,0.0,0.0,0.0,0.0,0.0,0.0,1.0


# Tokenize and encode using pytorch datasets


In [39]:
class ExplicitStereotypeDataset(Dataset):

  def __init__(self, data: pd.DataFrame, tokenizer,max_token_len: int = 50):
    self.tokenizer = tokenizer
    self.data = data
    self.max_token_len = max_token_len
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index: int):
    data_row = self.data.iloc[index]
    text = data_row[0]
    # labels = data_row[2]
    labels = list(data_row.iloc[1:].to_dict().values()) # To handle one-hot encoded categorical values [0-8] 

    encoding = self.tokenizer.encode_plus(
      text,
      add_special_tokens=True,
      max_length=self.max_token_len,
      return_token_type_ids=False,
      padding= True,
      truncation=True,
      return_attention_mask=True,
      return_tensors='pt',
    )

    return dict(
      # text=text,
      input_ids=encoding["input_ids"].flatten(),
      attention_mask=encoding["attention_mask"].flatten(),
      labels= torch.FloatTensor(labels)
    )

In [40]:
def create_train_val_datasets(tokenizer):

  train_dataset = ExplicitStereotypeDataset(train_df, tokenizer, max_token_len=MAX_LEN)
  val_dataset = ExplicitStereotypeDataset(val_df, tokenizer, max_token_len=MAX_LEN)
  test_dataset = ExplicitStereotypeDataset(test_df, tokenizer, max_token_len=MAX_LEN)

  return train_dataset, val_dataset, test_dataset

# Fine-tuning 

Fine-tuning for Multi-Label classification can be done by either 

1. Creating a model that overrides the `forward` method of huggingface transformers with 
  * Appropriate pooling
  * Loss function : `torch.nn.BCEWithLogitsLoss()`
2. Creating a custom `trainer` that overrides `compute_loss`

Reference:

1. Huggingface : https://colab.research.google.com/drive/1X7l8pM6t4VLqxQVJ23ssIxmrsc4Kpc5q?usp=sharing#scrollTo=XZEN8MhaL54M
2. https://github.com/gkebe/mlmc/blob/master/mlmc_class.py


Methodology :

* Using the pooling used in huggingface transformers for sequence classification.
  * gpt2 : **TBD after the LM notes section**
  * XLNet-base-cased : **TBD after the LM notes section**
* Overriding the `compute_loss` of `trainer` class. 


## XLnet, GPT-2

Method:
  * Creating a custom `trainer` that overrides `compute_loss`

In [41]:
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits
        loss_fct = torch.nn.BCEWithLogitsLoss()
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), 
                        labels.float().view(-1, self.model.config.num_labels))
        return (loss, outputs) if return_outputs else loss

# Training 

## Metrics

In [42]:
def Accuracy(y_true, y_pred):
  temp = 0
  for i in range(y_true.shape[0]):
      temp += sum(np.logical_and(y_true[i], y_pred[i])) / sum(np.logical_or(y_true[i], y_pred[i]))
  return temp / y_true.shape[0]

In [59]:
from sklearn.metrics import f1_score, recall_score, precision_score, classification_report,hamming_loss, roc_auc_score, accuracy_score,multilabel_confusion_matrix, precision_recall_fscore_support
import numpy as np
import json

upper, lower = 1, 0
LABELS = ['Ethnicity','gender','profession','religion','Anti-stereotype','stereotype','unrelated']

def classification_metrics(test_pred,labels,model_name,threshold):

  print("Evaluation metrics for test set:")
  y_pred = np.where(test_pred > threshold, upper, lower)

  ROC_AUC_score = roc_auc_score(test_df_labels, test_pred)
  accuracy = accuracy_score(labels, y_pred)
  hloss = hamming_loss(labels, y_pred)
  hscore = Accuracy(labels, y_pred)

  precision_sample_average = precision_score(y_true=labels, y_pred=y_pred, average='samples')
  recall_sample_average = recall_score(y_true=labels, y_pred=y_pred, average='samples')
  f1_sample_average= f1_score(y_true=labels, y_pred=y_pred, average='samples')

  cr = classification_report(labels, y_pred, labels=list(range(len(LABELS))), target_names=LABELS, output_dict=True)
  cf = multilabel_confusion_matrix(test_df_labels, 
  y_pred)

  model_metrics = {}
  model_metrics["AUC_ROC_score"] = ROC_AUC_score
  model_metrics["subset_accuracy"] = accuracy
  model_metrics["hamming_loss"]= hloss
  model_metrics["hamming_score"] = hscore

  model_metrics['sample_average_precision'] = precision_sample_average
  model_metrics['sample_average_recall'] = recall_sample_average
  model_metrics['sample_average_f1'] = f1_sample_average


  if write_to_file == True:
    model_metrics["Classification_report"] = cr

    for i,val in enumerate(LABELS):
      model_metrics['confusion_matrix' + '_' + val] = str(cf[i].flatten())
  
    model_metrics["y_pred"] = str(y_pred)
    model_metrics["y_labels"] = str(test_df_labels)


    if threshold != 0.5:
      th = "calculated_threshold"
    else:
      th = threshold

    model_metrics["threshold"] = th
    output_file = "eval_results_" + model_name + "_"+str(th) +"_"+ ".json"
    
    with open(output_file, "w" ) as writer:
        json.dump(model_metrics,writer)
    print("  Saving the metrics into a file: " + output_file + " with threshold :" + str(threshold))
  
  return model_metrics
  # print("\n ROC-AUC score: %.6f \n" % (ROC_AUC_score))
  # print("\n Subset accuracy : %.6f \n" % (accuracy))
  # print("\n hamming_loss : %.6f \n" % (hloss))
  # print("\n hamming score : %.6f \n" % hscore)
  # print("\n sample average  precision_sample_average : %.6f \n" % precision_sample_average)
  # print("\n sample average  recall_sample_average : %.6f \n" % recall_sample_average)
  # print("\n sample average  f1_sample_average : %.6f \n" % f1_sample_average)

In [56]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    return classification_metrics(predictions,labels,MODEL_NAME,0.5)

## XLNet

In [45]:
# Number of epochs 
N_EPOCHS = 2

# Batch_size 
BATCH_SIZE = 32

# Model name 
MODEL = 'xlnet-base-cased'

MODEL_NAME = 'xlnet'

# Learning rate 
learning_rate = 2.49816047538945e-05

# Number of labels 
num_labels = 7

In [46]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels).to('cuda')

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [47]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

In [48]:
batch_size = BATCH_SIZE

args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Trained_models/mult_label_dataset/Unrelated_samples_adjusted/Huggingface_XLNet_GPT-2/XLnet/stereotype_classification_XLNet",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=N_EPOCHS,
    weight_decay=0.01
)

In [49]:
train_dataset,val_dataset, test_dataset = create_train_val_datasets(tokenizer)

In [50]:
train_dataset

In [51]:
sample = train_dataset[0]

In [52]:
sample['labels']

tensor([0., 0., 0., 1., 0., 1., 0.])

In [60]:
multi_trainer = MultilabelTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

In [61]:
write_to_file = False # Disable logging the metrics to file 
multi_trainer.train()

Epoch,Training Loss,Validation Loss,Auc Roc Score,Subset Accuracy,Hamming Loss,Hamming Score,Sample Average Precision,Sample Average Recall,Sample Average F1,Runtime,Samples Per Second
1,No log,0.193340,0.498875,0.526570,0.098516,0.706320,0.872585,0.719807,0.770666,12.634900,196.599000
2,0.244300,0.174639,0.496916,0.595008,0.085116,0.752214,0.893720,0.767915,0.809850,12.736800,195.025000


Evaluation metrics for test set:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation metrics for test set:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=726, training_loss=0.21890421150144468, metrics={'train_runtime': 468.1485, 'train_samples_per_second': 1.551, 'total_flos': 626976480737376.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 4096, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 712704, 'train_mem_gpu_alloc_delta': 1418223616, 'train_mem_cpu_peaked_delta': 7782400, 'train_mem_gpu_peaked_delta': 1450475008})

In [ ]:
write_to_file = True # Disable logging the metrics to file 
multi_trainer.evaluate()

In [62]:
write_to_file = True # Enable logging the metrics to file 
multi_trainer.evaluate(test_dataset,metric_key_prefix="test")

Evaluation metrics for test set:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Trainer is attempting to log a value of "{'Ethnicity': {'precision': 0.96625, 'recall': 0.985969387755102, 'f1-score': 0.9760101010101009, 'support': 784}, 'gender': {'precision': 0.8951310861423221, 'recall': 0.7861842105263158, 'f1-score': 0.8371278458844135, 'support': 304}, 'profession': {'precision': 0.9014989293361885, 'recall': 0.9014989293361885, 'f1-score': 0.9014989293361885, 's

  Saving the metrics into a file: eval_results_xlnet_0.5_.json with threshold :0.5


{'epoch': 2.0,
 'eval_mem_cpu_alloc_delta': 4096,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 159477760,
 'test_AUC_ROC_score': 0.9650743405392912,
 'test_Classification_report': {'Anti-stereotype': {'f1-score': 0.5356223175965665,
   'precision': 0.8062015503875969,
   'recall': 0.40102827763496146,
   'support': 778},
  'Ethnicity': {'f1-score': 0.9760101010101009,
   'precision': 0.96625,
   'recall': 0.985969387755102,
   'support': 784},
  'gender': {'f1-score': 0.8371278458844135,
   'precision': 0.8951310861423221,
   'recall': 0.7861842105263158,
   'support': 304},
  'macro avg': {'f1-score': 0.8466299043854991,
   'precision': 0.9132213310057649,
   'recall': 0.8046776860357208,
   'support': 4332},
  'micro avg': {'f1-score': 0.8322231996983788,
   'precision': 0.9133793103448276,
   'recall': 0.7643120960295475,
   'support': 4332},
  'profession': {'f1-score': 0.9014989293361885,
   'precision': 0.9014989293361885,
   'rec

## GPT-2

In [66]:
# Number of epochs 
N_EPOCHS = 2

# Batch_size 
BATCH_SIZE = 32

# Model name 
MODEL = 'gpt2'

MODEL_NAME = 'gpt-2'

# Learning rate 
learning_rate = 2.49816047538945e-05

# Number of labels 
num_labels = 7

In [67]:
model = AutoModelForSequenceClassification.from_pretrained(MODEL, num_labels=num_labels).to('cuda')

Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [68]:
# model = GPT2ForMultiLabelSequenceClassification.from_pretrained(MODEL, num_labels=num_labels).to('cuda')

In [69]:
tokenizer = AutoTokenizer.from_pretrained(MODEL)

special_tokens_dict = {'pad_token': '[PAD]'}

# default to left padding
tokenizer.padding_side = "left"
# Define PAD Token = EOS Token = 50256
tokenizer.pad_token = tokenizer.eos_token

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [70]:
# resize model embedding to match new tokenizer
model.resize_token_embeddings(len(tokenizer))

# fix model padding token id
model.config.pad_token_id = model.config.eos_token_id

In [71]:
batch_size = BATCH_SIZE

args = TrainingArguments(
    output_dir="/content/drive/MyDrive/Trained_models/mult_label_dataset/Unrelated_samples_adjusted/Huggingface_XLNet_GPT-2/GPT-2",
    evaluation_strategy = "epoch",
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=N_EPOCHS,
    weight_decay=0.01
)

In [72]:
train_dataset,val_dataset, test_dataset = create_train_val_datasets(tokenizer)

In [73]:
multi_trainer = MultilabelTrainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer)

In [74]:
write_to_file = False # Disable logging the metrics to file 
multi_trainer.train()

Epoch,Training Loss,Validation Loss,Auc Roc Score,Subset Accuracy,Hamming Loss,Hamming Score,Sample Average Precision,Sample Average Recall,Sample Average F1,Runtime,Samples Per Second
1,No log,0.291762,0.499523,0.353060,0.147458,0.522410,0.685118,0.528784,0.580824,12.878300,192.883000
2,0.392900,0.261770,0.497469,0.367955,0.135093,0.569612,0.760937,0.579911,0.639801,13.037300,190.530000


Evaluation metrics for test set:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Evaluation metrics for test set:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=726, training_loss=0.35575091674636544, metrics={'train_runtime': 418.7589, 'train_samples_per_second': 1.734, 'total_flos': 604594526330880.0, 'epoch': 2.0, 'init_mem_cpu_alloc_delta': 0, 'init_mem_gpu_alloc_delta': 0, 'init_mem_cpu_peaked_delta': 0, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': -651014144, 'train_mem_gpu_alloc_delta': 1502584320, 'train_mem_cpu_peaked_delta': 651268096, 'train_mem_gpu_peaked_delta': 2036073472})

In [76]:
write_to_file = False # Enable logging the metrics to file 
multi_trainer.evaluate()

Evaluation metrics for test set:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 2.0,
 'eval_AUC_ROC_score': 0.4974693451115461,
 'eval_hamming_loss': 0.13509316770186336,
 'eval_hamming_score': 0.5696121846484158,
 'eval_loss': 0.2617700695991516,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 281281536,
 'eval_runtime': 13.2373,
 'eval_sample_average_f1': 0.639801395598497,
 'eval_sample_average_precision': 0.7609366612989801,
 'eval_sample_average_recall': 0.5799114331723028,
 'eval_samples_per_second': 187.651,
 'eval_subset_accuracy': 0.3679549114331723}

In [78]:
write_to_file = False # Enable logging the metrics to file 
multi_trainer.evaluate(test_dataset,metric_key_prefix="test")

Evaluation metrics for test set:


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


{'epoch': 2.0,
 'eval_mem_cpu_alloc_delta': 0,
 'eval_mem_cpu_peaked_delta': 0,
 'eval_mem_gpu_alloc_delta': 0,
 'eval_mem_gpu_peaked_delta': 281255424,
 'test_AUC_ROC_score': 0.9142304301507308,
 'test_hamming_loss': 0.1324476650563607,
 'test_hamming_score': 0.5717257112184643,
 'test_loss': 0.2523163855075836,
 'test_runtime': 13.69,
 'test_sample_average_f1': 0.6439076757917338,
 'test_sample_average_precision': 0.7701959205582394,
 'test_sample_average_recall': 0.5813204508856683,
 'test_samples_per_second': 181.446,
 'test_subset_accuracy': 0.3635265700483092}

In [ ]:
write_to_file = True # Enable logging the metrics to file 
multi_trainer.evaluate(test_dataset,metric_key_prefix="test")